In [1]:
# RUN THIS
import pandas as pd

In [2]:
# RUN THIS (takes 30 sec)
# Load CSV's
neov_df = pd.read_csv('../neo4j-import/out/neo_ver.csv')
nu_df = pd.read_csv('../../samples/neo4j-cross/multiple-ecosystems-results.csv')
old_df = pd.read_csv('out/non_unique_repo.csv')


In [3]:
# RUN THIS
# Get unique package managers and how many times they appear
list_pms = []
for pms in nu_df["pkgmans"]:
    for pm in pms[1:-1].split(','):
        list_pms.append(pm)
print(len(list_pms))
q = pd.DataFrame(list_pms)
q.value_counts()

4552


freebsd12     1015
netbsd9        768
vcpkg          600
conan          543
openbsd69      471
rubygems       437
npm            215
cargo          210
homebrew       123
chromebrew     120
clojars         42
alire            3
luarocks         3
fpm              1
nimble           1
dtype: int64

In [17]:
# RUN THIS (get data for \label{tbl:cross-pkgs})
# Get packages managers that appear with given package manager

def get_pkgmans(pkgmans):
    res_df = pd.DataFrame(columns=('pkgman', 'has_cross_pkgs_in'))
    for pkgman in pkgmans:
        pkgs = nu_df[nu_df['pkgmans'].str.contains(pkgman)]
        pmns = []
        for pms in pkgs['pkgmans']:
            for pm in pms[1:-1].split(','):
                pmns.append(pm)
        pmns = list(set(pmns))
        pmns.remove(pkgman)
        pmns.sort()
        entry = pd.DataFrame.from_dict({
                "pkgman": [pkgman],
                "has_cross_pkgs_in": [pmns],
            })
        res_df = pd.concat([res_df, entry], ignore_index=True)
    return res_df

pkgmanagers = [
    'rubygems',
    "npm",
    "freebsd12",
    "netbsd9",
    "clojars",
    "vcpkg",
    "conan",
    "openbsd69",
    "cargo",
    "chromebrew",
    "homebrew",
    "luarocks",
    "alire",
    "fpm",
    "nimble"
]
result = get_pkgmans(pkgmanagers)
result.to_csv('out/cross_pkgman.csv', index=False)

result



,pkgman,has_cross_pkgs_in
0,rubygems,"[cargo, clojars, conan, freebsd12, netbsd9, np..."
1,npm,"[cargo, clojars, conan, freebsd12, homebrew, l..."
2,freebsd12,"[cargo, clojars, conan, fpm, luarocks, netbsd9..."
3,netbsd9,"[cargo, conan, freebsd12, luarocks, npm, openb..."
4,clojars,"[cargo, freebsd12, npm, openbsd69, rubygems]"
5,vcpkg,"[alire, cargo, conan, freebsd12, netbsd9, npm,..."
6,conan,"[alire, cargo, chromebrew, freebsd12, homebrew..."
7,openbsd69,"[cargo, clojars, conan, freebsd12, netbsd9, np..."
8,cargo,"[clojars, conan, freebsd12, netbsd9, npm, open..."
9,chromebrew,"[conan, homebrew]"


In [4]:
# NEW SCRIPT
# This script generates a dataframe that contains a list of packages, sorted with the most reverse dependencies. Also prints the total number of unique packages
from ast import literal_eval
data_path = '../../dasea_data'
path_dict = {
    'alire': '/alire/alire_dependencies_05-17-2022.csv',
    'cargo': '/cargo/cargo_dependencies_05-17-2022.csv',
    'chromebrew': '/chromebrew/chromebrew_dependencies_05-17-2022.csv',
    'clojars': '/clojars/clojars_dependencies_05-19-2022.csv',
    'conan': '/conan/conan_dependencies_05-17-2022.csv',
    'freebsd12': '/ports/freebsd12/freebsd12_dependencies_05-17-2022.csv',
    'fpm': '/fpm/fpm_dependencies_05-17-2022.csv',
    'homebrew': '/homebrew/homebrew_dependencies_05-17-2022.csv',
    'netbsd9': '/ports/netbsd9/netbsd9_dependencies_05-17-2022.csv',
    'nimble': '/nimble/nimble_dependencies_05-17-2022.csv',
    'openbsd69': '/ports/openbsd69/openbsd69_dependencies_05-17-2022.csv',
    'vcpkg': '/vcpkg/vcpkg_dependencies_05-17-2022.csv',
    'luarocks': '/luarocks/luarocks_dependencies_05-17-2022.csv',
    'npm': '/npm/npm_dependencies_05-17-2022.csv',
    'rubygems': '/rubygems/rubygems_dependencies_05-17-2022.csv'
}

df_dict = {}

def _df(path):
    import os
    file_path = os.path.abspath(data_path + path)
    return pd.read_csv(file_path)

def string_to_list_of_dicts(string):
    return literal_eval(string.replace('":', '":"').replace(',"','","').replace("}", '"}'))

def is_valid(dictionary):
    zero_count = 0
    for key in dictionary:
        if dictionary[key] == 0:
            zero_count += 1
    return len(dictionary) - zero_count > 1

def sum_all_values_in_dict(dictionary):
    return sum(dictionary.values())

res_df = pd.DataFrame(columns=('url', 'count_deps', 'count_total'))
counter = 0

df = pd.read_csv("../../samples/neo4j-cross/multiple-ecosystems-results.csv")

pck_count = 0
for row in df.itertuples():
    counter += 1
    print(f'{counter}/{len(df)}')
    # if counter > 10:
    #     break
    url = row[2]
    if "git" not in url: # skip URLs that is not git
        continue
    names_and_pkgmans = string_to_list_of_dicts(row[1])
    pkgmans = []
    res = {}
    for name_and_pkgman in names_and_pkgmans:
        pck_count += 1
        name = name_and_pkgman['name']
        pkgman = name_and_pkgman['pkgman']
        pkgmans.append(pkgman)
        if pkgman not in res:
            res[pkgman] = 0
        
        path = path_dict[pkgman]
        ddf = pd.DataFrame()
        if path in df_dict:
            ddf = df_dict[path]
        else:
            ddf = _df(data_path + path)
            df_dict[path] = ddf
        targets = ddf[ddf['target_name'] == name]
        uniq = targets['source_name'].unique()
        # print(uniq)
        if len(uniq) > 0:
            res[pkgman] = res[pkgman] + len(uniq)
    if is_valid(res):
        entry = pd.DataFrame.from_dict({
            "url": [url],
            "count_deps": [res],
            "count_total": [sum(res.values())]
        })
        res_df = pd.concat([res_df, entry], ignore_index=True)

print(pck_count)



1/2090
2/2090
3/2090
4/2090
5/2090
6/2090
7/2090
8/2090
9/2090
10/2090
11/2090
191


In [5]:
# RUN THIS to export previous dataframe to csv
# res_df.sort_values(by=['count_total'], ascending=False).to_csv('out/most_reverse_deps.csv')
res_df.sort_values(by=['count_total'], ascending=False).to_json('out/most_reverse_deps2.json')